# Decoder Transformer for Text Generation

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

## Preprocessing


In [2]:
# Read in the txt file using the latin1 encoding
input_file_path = os.path.join('../data', 'hamlet.txt')
with open(input_file_path, 'r', encoding='latin1') as f:
    text = f.read()
    #text.encode('utf-8').strip()
    print(text[:150])
print(f"length of text: {len(text)}")

Title: Hamlet, Prince of Denmark

Author: William Shakespeare

Editor: Charles John Kean

Release date: January 10, 2009 [eBook #27761]
              
length of text: 221770


In [3]:
# Create a list with all the characters and get the size of it
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

all the unique characters:  
 !"#$%&'()*,-./0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{} ¡¢¦¨©­³ÃÎÙÚâ
vocab size: 120


In [4]:
# Create a mapping from string ot integer and from integer to string
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] 
def decode(l):
    return ''.join([itos[i] for i in l]) 

In [5]:
# Encode the entire text and store it in a tensor object
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) 

torch.Size([221770]) torch.int64
tensor([59, 77, 88, 80, 73, 35, 10, 47, 69, 81, 80, 73, 88, 21, 10, 55, 86, 77,
        82, 71, 73, 10, 83, 74, 10, 43, 73, 82, 81, 69, 86, 79,  5,  5, 40, 89,
        88, 76, 83, 86, 35, 10, 62, 77, 80, 80, 77, 69, 81, 10, 58, 76, 69, 79,
        73, 87, 84, 73, 69, 86, 73,  5,  5, 44, 72, 77, 88, 83, 86, 35, 10, 42,
        76, 69, 86, 80, 73, 87, 10, 49, 83, 76, 82, 10, 50, 73, 69, 82,  5,  5,
        57, 73, 80, 73, 69, 87, 73, 10, 72, 69, 88, 73, 35, 10, 49, 69, 82, 89,
        69, 86, 93, 10, 26, 25, 21, 10, 27, 25, 25, 34, 10, 66, 73, 41, 83, 83,
        79, 10, 13, 27, 32, 32, 31, 26, 67,  5, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 52, 83, 87, 88, 10, 86, 73, 71, 73, 82,
        88, 80, 93, 10, 89, 84, 72, 69, 88, 73, 72, 35, 10, 49, 69, 82, 89, 69,
        86, 93, 10, 29, 21, 10, 27, 25, 27, 26,  5,  5, 51, 69, 82, 75, 89, 69,
        75, 73, 35, 10, 44, 82, 75, 80, 77, 87, 76,  5,  5, 42, 86, 73, 72, 77,
       

In [6]:
# Split the training into test and validation sets
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]
print(f"train has {len(train_data):,} entries")
print(f"val has {len(val_data):,} entries")

train has 199,593 entries
val has 22,177 entries


In [7]:
# Define the size of one input batch
block_size = 8
print("An example block: ", train_data[:block_size+1])

# Retrieve the traget letter when the input sequence is known, i.e. show the next character
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Input of the tensor is: {context} & the target: {target}")

An example block:  tensor([59, 77, 88, 80, 73, 35, 10, 47, 69])
Input of the tensor is: tensor([59]) & the target: 77
Input of the tensor is: tensor([59, 77]) & the target: 88
Input of the tensor is: tensor([59, 77, 88]) & the target: 80
Input of the tensor is: tensor([59, 77, 88, 80]) & the target: 73
Input of the tensor is: tensor([59, 77, 88, 80, 73]) & the target: 35
Input of the tensor is: tensor([59, 77, 88, 80, 73, 35]) & the target: 10
Input of the tensor is: tensor([59, 77, 88, 80, 73, 35, 10]) & the target: 47
Input of the tensor is: tensor([59, 77, 88, 80, 73, 35, 10, 47]) & the target: 69


In [8]:
# Use a random seed to get a sequence and set the batch size that shows how many independent sequences we process in parallel
torch.manual_seed(1337)
batch_size = 4 

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    if split == 'train':
        data = train_data
    else:
        data = val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Create a tensor with the size of batch_size and select a random int between 0-(len(data)-block_size) for every position 
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[21, 10, 16, 69, 81, 84, 36, 71],
        [93, 10, 80, 83, 86, 72, 23,  5],
        [87, 36,  5, 10, 10, 62, 73, 17],
        [73, 10, 69, 10, 75, 83, 83, 72]])
targets:
torch.Size([4, 8])
tensor([[10, 16, 69, 81, 84, 36, 71, 23],
        [10, 80, 83, 86, 72, 23,  5,  5],
        [36,  5, 10, 10, 62, 73, 17, 80],
        [10, 69, 10, 75, 83, 83, 72, 10]])
----
when input is [21] the target: 10
when input is [21, 10] the target: 16
when input is [21, 10, 16] the target: 69
when input is [21, 10, 16, 69] the target: 81
when input is [21, 10, 16, 69, 81] the target: 84
when input is [21, 10, 16, 69, 81, 84] the target: 36
when input is [21, 10, 16, 69, 81, 84, 36] the target: 71
when input is [21, 10, 16, 69, 81, 84, 36, 71] the target: 23
when input is [93] the target: 10
when input is [93, 10] the target: 80
when input is [93, 10, 80] the target: 83
when input is [93, 10, 80, 83] the target: 86
when input is [93, 10, 80, 83, 86] the target: 72
when inp

## Bigram Model


In [9]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) -> Batch, Tokens/Time, Channel <=> 4,8,120

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 120])
tensor(5.0029, grad_fn=<NllLossBackward0>)
 &³h]zyGf-y{1y/D[<Y<E,D)>QA)vr.41c2eP¢d<(#Yh<;DB6©y<; .©¦>¦sd¡t5 [c3a1pVK#'g;T4"Úz


In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.6345016956329346


In [12]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 Lo cha tes aboeclo  poret Co onoue f at Pve oofrm. tay, s  T&Jof?
 chont   hyongllur._ _cir_ _Hat ANot t ndutho in86:
Equtrckisth,  I _,  _Haveap.]
RRe apoby;

bundea I.[Fou. O tst,

 e Tiguspouray atotrad  s m>'stus  alis  _ tham.]  ORGour is I.]
 l,
  _  ys,  moit ot 100]
harerm.
 he   with.C.nerthise pot woruPashe  it tred Af

 pe:---akendin  Call onfar---qwhes ore Whealath
 he DONaco an ted tothal atexorefin PH.
 Whotothtonenthil
 t.] me me her (_ welat.] terathavig.8 Bure, jclon! rerechae


## Self Attention

In [13]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

## Normalization 

In [14]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

## Final Model - Bigram Model with Self-Attention - a Transformer

In [16]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 200
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))

0.216824 M parameters
step 0: train loss 4.9881, val loss 4.9843
step 100: train loss 2.8239, val loss 3.1097
step 199: train loss 2.5505, val loss 2.9908
 o frelare,
   _bscpicinwe beded she:
   _   [_   A _¢_Hen   mamo d _hebg,_ ady p( are m'sengnd]

e ?
